# Gaining familiarity with web3py

In this notebook I will work through the examples provided in the [web3py documentation](https://web3py.readthedocs.io/en/stable/examples.html) to get a better feeling for the functionality. My goal is to be able to compile, deploy and interact with Solidity contracts directly from within Python.

### Software used

Over the last two days I set up LuceVM, an Ubuntu 16.04 virtual machine that contains all the required frameworks. The most important tools for us are:
* Python 3.7
    * web3
    * solc-py-x
* Jupyter Notebook for interaction and documentation
* Virtual Environments via conda to handle dependencies
* Solidity Compiler
* Node.js
* Ganache

### Preparation: Connect Python to local Ganache Node

First we start an instance of Ganache, either via GUI or the  ```run_ganache.sh``` script in LuceVM.

In [ ]:
from web3 import Web3, HTTPProvider

# Ganache Connection
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

### Obtain Accounts

In [ ]:
# Extract default accounts from ganache
accounts = w3.eth.accounts

In [ ]:
# Display address of first account
accounts[0]

Store credentials of first account for testing:

In [ ]:
# Wallet address
wallet_address       = "0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA"

# Private key (from Ganache interface)
wallet_private_key   = "0x4a2cb86c7d3663abebf7ab86a6ddc3900aee399750f35e65a44ecf843ec39116"

### Send ether

In [ ]:
import time
def send_ether(amount_in_ether, recipient_address):
    amount_in_wei = w3.toWei(amount_in_ether,'ether');

    # How many transactions have been made by wallet?
    nonce = w3.eth.getTransactionCount(wallet_address)
    
    # Specify transcation details
    txn_dict = {
            'to': recipient_address,
            'value': amount_in_wei,
            'gas': 2000000,
            'gasPrice': w3.toWei('40', 'gwei'),
            'nonce': nonce,
            'chainId': 3
    }
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(txn_dict, wallet_private_key)

    # Send transaction & store transaction hash
    txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

    # Check if transaction was added to blockchain
    time.sleep(0.5)
    txn_receipt = w3.eth.getTransactionReceipt(txn_hash)
    return txn_hash

In [ ]:
# Set recipient
recipient = accounts[1]

In [ ]:
# Send ether and store transaction hash
txn_hash = send_ether(1,recipient)

In [ ]:
# WHY DOES THIS NOT WORK?
# It is part of the function call in send ether and seems to work there..
# w3.eth.getBlockTransactionCount(0x92d44e8579620f2db88a12e70fe38e8cdb3541ba)

### Look up a block

In [ ]:
w3.eth.getBlock(1)

In [ ]:
# Retrieve the last block
w3.eth.getBlock('latest')

In [ ]:
# directly obtain the current block number
w3.eth.blockNumber

### Loop up a transaction via hash

In [ ]:
w3.eth.getTransaction(txn_hash)

In [ ]:
# Transaction receipt
w3.eth.getTransactionReceipt(txn_hash)

---

In [ ]:
pwd

### Compile contract

Important that py-solc is installed for code to execute.

In [2]:
#!{sys.executable} -m pip install py-solc

In [4]:
#!{sys.executable} -m pip install py-solc-x

In [10]:
import sys
import time
import pprint

from web3.providers.eth_tester import EthereumTesterProvider
from web3 import Web3
from solcx import compile_source


def compile_source_file(file_path):
    with open(file_path, 'r') as f:
        source = f.read()

    return compile_source(source)

Using solc version v0.4.25


In [11]:
pwd

'/vagrant/jupyter'

In [12]:
ls

0) jupyter_kernel_test.ipynb  2) hello_luce.ipynb
1) Web3 Examples.ipynb        data/


In [13]:
contract_source_path = './data/contract.sol'

In [14]:
compiled_sol = compile_source_file('./data/contract.sol')

In [15]:
contract_id, contract_interface = compiled_sol.popitem()

In [16]:
contract_id

'<stdin>:StoreVar'

In [17]:
# Interface contains ABI and more
contract_interface

{'abi': [{'constant': True,
   'inputs': [],
   'name': '_myVar',
   'outputs': [{'name': '', 'type': 'uint8'}],
   'payable': False,
   'stateMutability': 'view',
   'type': 'function'},
  {'constant': True,
   'inputs': [],
   'name': 'getVar',
   'outputs': [{'name': '', 'type': 'uint8'}],
   'payable': False,
   'stateMutability': 'view',
   'type': 'function'},
  {'constant': False,
   'inputs': [{'name': '_var', 'type': 'uint8'}],
   'name': 'setVar',
   'outputs': [],
   'payable': False,
   'stateMutability': 'nonpayable',
   'type': 'function'},
  {'anonymous': False,
   'inputs': [{'indexed': True, 'name': '_var', 'type': 'uint256'}],
   'name': 'MyEvent',
   'type': 'event'}],
 'asm': {'.code': [{'begin': 0, 'end': 262, 'name': 'PUSH', 'value': '80'},
   {'begin': 0, 'end': 262, 'name': 'PUSH', 'value': '40'},
   {'begin': 0, 'end': 262, 'name': 'MSTORE'},
   {'begin': 0, 'end': 262, 'name': 'CALLVALUE'},
   {'begin': 8, 'end': 17, 'name': 'DUP1'},
   {'begin': 5, 'end': 7, 

Up to this point everything is working. We can compile the contract and also have the abi and binary code conveniently stored in our ```contract_interface```.

---

### Deploy Contract

Attempt to deploy 1:  
Fails

In [ ]:
abi = contract_interface['abi']
bytecode = contract_interface['bin']

In [ ]:
tx_hash = w3.eth.contract(abi,bytecode).deploy()

Attempt to deploy 2: (exactly as per documentation)    
Fails

In [ ]:
def deploy_contract(w3, contract_interface):
    tx_hash = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']).deploy()

    address = w3.eth.getTransactionReceipt(tx_hash)['contractAddress']
    return address

In [ ]:
address = deploy_contract(w3, contract_interface)

In [ ]:
print("Deployed {0} to: {1}\n".format(contract_id, address))

---

### Remaining Part of Example Section

In [ ]:
def wait_for_receipt(w3, tx_hash, poll_interval):
    while True:
        tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
        if tx_receipt:
            return tx_receipt
        time.sleep(poll_interval)

In [ ]:
contract_interface['abi']

In [ ]:
address = deploy_contract(w3, contract_interface)

In [ ]:
print("Deployed {0} to: {1}\n".format(contract_id, address))

In [ ]:
store_var_contract = w3.eth.contract(
   address=address,
   abi=contract_interface['abi'])

gas_estimate = store_var_contract.functions.setVar(255).estimateGas()
print("Gas estimate to transact with setVar: {0}\n".format(gas_estimate))

if gas_estimate < 100000:
    print("Sending transaction to setVar(255)\n")
    tx_hash = store_var_contract.functions.setVar(255).transact()
    receipt = wait_for_receipt(w3, tx_hash, 1)
    print("Transaction receipt mined: \n")
    pprint.pprint(dict(receipt))
else:
    print("Gas cost exceeds 100000")

### Deployment Attempt 2
Since the old way to deploy contracts via `w3.eth.contract.deploy()` seems to be deprecated I try out another pathway, as outlined in the contract section of the web3.py documentation [here](https://web3py.readthedocs.io/en/stable/contracts.html?highlight=abi).

Note: I replace solc with solcx since solc is throwing errors during compilation. solcx can compile the contract.

In [ ]:
#!{sys.executable} -m pip install -U web3[tester]

There is a requirement conflict with eth-abi when trying to install web3tester. See if we can use ganache as sandbox instead.

In [23]:
import json
import web3

from web3 import Web3
from solcx import compile_source
from web3.contract import ConciseContract

In [24]:
# Solidity source code
contract_source_code = '''
pragma solidity ^0.4.21;

contract Greeter {
    string public greeting;

    function Greeter() public {
        greeting = 'Hello';
    }

    function setGreeting(string _greeting) public {
        greeting = _greeting;
    }

    function greet() view public returns (string) {
        return greeting;
    }
}
'''

In [32]:
compiled_sol = compile_source(contract_source_code) # Compiled source code
contract_interface = compiled_sol['<stdin>:Greeter']

The compilation works fine but only when using solcx instead of solc.

In [31]:
# web3.py instance
# w3 = Web3(Web3.EthereumTesterProvider())
# This causes a requirement conflix with eth-abi.. Try to use Ganache instead

In [35]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [38]:
# set pre-funded account as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

In [39]:
# Instantiate and deploy contract
Greeter = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

In [40]:
# Submit the transaction that deploys the contract
tx_hash = Greeter.constructor().transact()

Halleluhjah! This worked :D  
The contract is deployed.

In [41]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [42]:
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xe7459bbf5955a416dd9dd59d494bb935b9e0563dc84fe5938817bcfe6b5bd1c2'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x4320602cb28f3b7cd403a95229bb54b2379fac91a09ded50380f80a21201a01d'),
 'blockNumber': 3,
 'from': '0x92d44e8579620f2db88a12e70fe38e8cdb3541ba',
 'to': None,
 'gasUsed': 364124,
 'cumulativeGasUsed': 364124,
 'contractAddress': '0xb88966F5cEc4612495F0B9D42b0b4Ad5B0Cf6329',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [43]:
# Create the contract instance with the newly-deployed address
greeter = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [44]:
# Display the default greeting from the contract
print('Default contract greeting: {}'.format(
    greeter.functions.greet().call()
))

Default contract greeting: Hello


In [45]:
greeter.functions.abi

[{'constant': False,
  'inputs': [{'name': '_greeting', 'type': 'string'}],
  'name': 'setGreeting',
  'outputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greet',
  'outputs': [{'name': '', 'type': 'string'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greeting',
  'outputs': [{'name': '', 'type': 'string'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'constructor'}]

In [49]:
greeter.functions.greet().call()

'Hello'

In [50]:
print('Setting the greeting to Nihao...')
tx_hash = greeter.functions.setGreeting('Nihao').transact()

Setting the greeting to Nihao...


In [51]:
# Wait for transaction to be mined...
w3.eth.waitForTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0x993f2dfe593f3ac17d448ed5bc62398d508e2826f35d04e0f176748c2c582c51'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xa536d7048b29425f89d0e8b9d59e6c1dbc25e8e6aee0df581685e093965e95f6'),
 'blockNumber': 4,
 'from': '0x92d44e8579620f2db88a12e70fe38e8cdb3541ba',
 'to': '0xb88966f5cec4612495f0b9d42b0b4ad5b0cf6329',
 'gasUsed': 33068,
 'cumulativeGasUsed': 33068,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')

In [52]:
greeter.functions.greet().call()

'Nihao'

All is working as intended! I am very happy. This gives enough foundation and the examples here can be used as reference for how to interact with LUCE contract functions.

### Drop Things Here

In [ ]:
# Contract address from Ganache
contract_address     = "0x9B3da536bfFf54974AE3D9151D7C6F5dBE81990E"